In [2]:
from genai_evaluator.clients import AWSBedrockLLMClient, TemplateStore

llm_client = AWSBedrockLLMClient(model="anthropic.claude-3-5-sonnet-20240620-v1:0")
template_store = TemplateStore(dir_path="genai_evaluator/templates", do_validation=True)

Loading templates from genai_evaluator\templates


In [3]:
from genai_evaluator.flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_ret)

SSLError: SSL validation failed for https://bedrock-runtime.eu-central-1.amazonaws.com/model/anthropic.claude-3-5-sonnet-20240620-v1%3A0/converse [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1018)

In [ ]:
import os

from promptflow.tracing import start_trace

from genai_evaluator.flows.generation_eval_flow import generation_eval_flow

os.environ["PF_DISABLE_TRACING"] = "FALSE"
start_trace(collection="trace_rag_metrics")

dict_eval_gen = generation_eval_flow(
    question="What is the capital of France?",
    answer_gt="The capital of France is Paris.",
    answer_pred="France's capital city is Paris, known for the Eiffel Tower.",
    context=[
        "Paris is the capital and most populous city of France.",
        "The Eiffel Tower, located in Paris, is a famous landmark.",
        "France is a country in Western Europe known for its art, culture, and cuisine.",
        "Lyon and Marseille are also major French cities but not the capital.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)

print(dict_eval_gen)

In [ ]:
from genai_evaluator.flows.retrieval_eval_flow import retrieval_eval_flow

dict_eval_ret = retrieval_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_ret)

In [ ]:
import os

from promptflow.tracing import start_trace

os.environ["PF_DISABLE_TRACING"] = "FALSE"


start_trace()
dict_eval_gen = generation_eval_flow(
    question="What are the main ingredients in traditional Italian pesto?",
    answer_gt="Traditional Italian pesto is made with basil, garlic, pine nuts, Parmesan cheese, and olive oil.",
    answer_pred="Italian pesto is usually made with spinach, almonds, cheddar cheese, and olive oil.",
    context=[
        "Pesto alla Genovese, the traditional Italian pesto, is made with fresh basil leaves, garlic, pine nuts, Parmesan cheese, and extra virgin olive oil.",
        "Spinach and other ingredients may be used in variations of pesto, but they are not part of the traditional recipe.",
        "The use of almonds or cheddar cheese is not typical in authentic Italian pesto recipes.",
        "Sunflower oil is not used in traditional Italian pesto.",
    ],
    llm_client=llm_client,
    template_store=template_store,
)
print(dict_eval_gen)